# Chapter 7: Grouping for Aggregation, Filtration and Transformation
## Recipes
* [Defining an aggregation](#Defining-an-aggregation)
* [Grouping and aggregating with multiple columns and functions](#Grouping-and-aggregating-with-multiple-columns-and-functions)
* [Removing the MultiIndex after grouping](#Removing-the-MultiIndex-after-grouping)
* [Customizing an aggregation function](#Customizing-an-aggregation-function)
* [Customizing aggregating functions with \*args and \*\*kwargs](#Customizing-aggregating-functions-with-*args-and-**kwargs)
* [Examining the groupby object](#Examining-a-groupby-object)
* [Filtering for states with a minority majority](#Filtering-for-states-with-a-minority-majority)
* [Transforming through a weight loss bet](#Transforming-through-a-weight-loss)
* [Calculating weighted mean SAT scores per state with apply](#Calculating-weighted-mean-SAT-scores-per-state-with-apply)
* [Grouping by continuous variables](#Grouping-by-continuous-variables)
* [Counting the total number of flights between cities](#Counting-the-total-number-of-flights-between-cities)
* [Finding the longest streak of on-time flights](#Finding-the-longest-streak-of-on-time-flights)

In [3]:
import pandas as pd
import numpy as np

# Defining an aggregation

In [4]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [5]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg('mean').head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [6]:
flights.groupby('AIRLINE').agg({'ARR_DELAY':'mean'}).head()

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580


In [7]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.mean).head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

In [8]:
flights.groupby('AIRLINE')['ARR_DELAY'].mean().head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

## How it works...

In [9]:
grouped = flights.groupby('AIRLINE')
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

## There's more

In [10]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.sqrt)

/Users/nickbarber/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:844: RuntimeWarning: invalid value encountered in sqrt
  f = lambda x: func(x, *args, **kwargs)
/Users/nickbarber/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/generic.py:907: RuntimeWarning: invalid value encountered in sqrt
  output = func(group, *args, **kwargs)


Exception: Must produce aggregated value

# Grouping and aggregating with multiple columns and functions

In [ ]:
flights = pd.read_csv('data/flights.csv')
flights.head()

In [ ]:
# The number of cancelled flights for every airline per day weekday
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED'].agg('sum').head(7)

In [ ]:
#Find the number and percentage of cancelled and diverted flights for every airline per weekday
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED', 'DIVERTED'].agg(['sum', 'mean']).head(7)

In [ ]:
# For each origin to destination flight, find the total number of flights, 
# the number and percentage of cancelled flights and the average and variance of the airtime. 
group_cols = ['ORG_AIR', 'DEST_AIR']
agg_dict = {'CANCELLED':['sum', 'mean', 'size'], 
            'AIR_TIME':['mean', 'var']}
flights.groupby(group_cols).agg(agg_dict).head()
# flights.groupby(['ORG_AIR', 'DEST_AIR']).agg({'CANCELLED': ['sum', 'mean', 'size'], 
#                                               'AIR_TIME':['mean', 'var']}).head()

# Removing the MultiIndex after grouping

In [ ]:
flights = pd.read_csv('data/flights.csv')
flights.head()

In [ ]:
airline_info = flights.groupby(['AIRLINE', 'WEEKDAY'])\
                      .agg({'DIST':['sum', 'mean'], 
                                    'ARR_DELAY':['min', 'max']}).astype(int)
airline_info.head()

In [ ]:
level0 = airline_info.columns.get_level_values(0)
level0

In [ ]:
level1 = airline_info.columns.get_level_values(1)
level1

In [ ]:
airline_info.columns = level0 + '_' + level1

In [ ]:
airline_info.head(7)

In [ ]:
airline_info.reset_index().head(7)

## There's more...

In [ ]:
flights.groupby(['AIRLINE'], as_index=False)['DIST'].agg('mean').round(0)

In [ ]:
flights.groupby(['AIRLINE'], as_index=False, sort=False)['DIST'].agg('mean')

# Customizing an aggregation function

In [ ]:
college = pd.read_csv('data/college.csv')
college.head()

In [ ]:
college.groupby('STABBR')['UGDS'].agg(['mean', 'std']).round(0).head()

In [ ]:
def max_deviation(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()

In [ ]:
college.groupby('STABBR')['UGDS'].agg(max_deviation).round(1).head()

## There's more...

In [ ]:
college.groupby('STABBR')['UGDS', 'SATVRMID', 'SATMTMID'].agg(max_deviation).round(1).head()

In [ ]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATVRMID', 'SATMTMID']\
       .agg([max_deviation, 'mean', 'std']).round(1).head()

In [ ]:
max_deviation.__name__

In [ ]:
max_deviation.__name__ = 'Max Deviation'

In [ ]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATVRMID', 'SATMTMID']\
       .agg([max_deviation, 'mean', 'std']).round(1).head()

# Customizing aggregating functions with \*args and \*\*kwargs

In [ ]:
college = pd.read_csv('data/college.csv')
grouped = college.groupby(['STABBR', 'RELAFFIL'])

In [ ]:
import inspect
inspect.signature(grouped.agg)

## How to do it...

In [ ]:
def pct_between_1_3k(s):
    return s.between(1000, 3000).mean()

In [ ]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between_1_3k).head(9)

In [ ]:
def pct_between(s, low, high):
    return s.between(low, high).mean()

In [ ]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between, 1000, 10000).head(9)

## How it works...

In [ ]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between, high=10000, low=1000).head(9)

In [ ]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between, 1000, high=10000).head(9)

## There's more...

In [ ]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(['mean', pct_between], low=100, high=1000)

In [ ]:
def make_agg_func(func, name, *args, **kwargs):
    def wrapper(x):
        return func(x, *args, **kwargs)
    wrapper.__name__ = name
    return wrapper

my_agg1 = make_agg_func(pct_between, 'pct_1_3k', low=1000, high=3000)
my_agg2 = make_agg_func(pct_between, 'pct_10_30k', 10000, 30000)

In [ ]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(['mean', my_agg1, my_agg2]).head()

# Examining a groupby object

In [ ]:
college = pd.read_csv('data/college.csv')
grouped = college.groupby(['STABBR', 'RELAFFIL'])
type(grouped)

In [ ]:
print([attr for attr in dir(grouped) if not attr.startswith('_')])

In [ ]:
grouped.ngroups

In [ ]:
groups = list(grouped.groups.keys())
groups[:6]

In [ ]:
grouped.get_group(('FL', 1)).head()

In [ ]:
from IPython.display import display

In [ ]:
i = 0
for name, group in grouped:
    print(name)
    display(group.head(2))
    i += 1
    if i == 5:
        break

In [ ]:
grouped.head(2).head(6)

## There's more...

In [ ]:
grouped.nth([1, -1]).head(8)

# Filtering for states with a minority majority

In [ ]:
college = pd.read_csv('data/college.csv', index_col='INSTNM')
grouped = college.groupby('STABBR')
grouped.ngroups

In [ ]:
%timeit college['STABBR'].nunique()

In [ ]:
def check_minority(df, threshold):
    minority_pct = 1 - df['UGDS_WHITE']
    total_minority = (df['UGDS'] * minority_pct).sum()
    total_ugds = df['UGDS'].sum()
    total_minority_pct = total_minority / total_ugds
    return total_minority_pct > threshold

In [ ]:
college_filtered = grouped.filter(check_minority, threshold=.5)
college_filtered.head()

In [ ]:
college.shape

In [ ]:
college_filtered.shape

In [ ]:
college_filtered['STABBR'].nunique()

## There's more...

In [ ]:
college_filtered_20 = grouped.filter(check_minority, threshold=.2)
college_filtered_20.shape

In [ ]:
college_filtered_20['STABBR'].nunique()

In [ ]:
college_filtered_70 = grouped.filter(check_minority, threshold=.7)
college_filtered_70.shape

In [ ]:
college_filtered_70['STABBR'].nunique()

In [ ]:
college_filtered_95 = grouped.filter(check_minority, threshold=.95)
college_filtered_95.shape

# Transforming through a weight-loss 

In [ ]:
weight_loss = pd.read_csv('data/weight_loss.csv')
weight_loss.query('Month == "Jan"')

In [ ]:
def find_perc_loss(s):
    return (s - s.iloc[0]) / s.iloc[0]

In [ ]:
bob_jan = weight_loss.query('Name=="Bob" and Month=="Jan"')
find_perc_loss(bob_jan['Weight'])

In [ ]:
pcnt_loss = weight_loss.groupby(['Name', 'Month'])['Weight'].transform(find_perc_loss)
pcnt_loss.head(8)

In [ ]:
weight_loss['Perc Weight Loss'] = pcnt_loss.round(3)
weight_loss.query('Name=="Bob" and Month in ["Jan", "Feb"]')

In [ ]:
week4 = weight_loss.query('Week == "Week 4"')
week4

In [ ]:
winner = week4.pivot(index='Month', columns='Name', values='Perc Weight Loss')
winner

In [ ]:
winner['Winner'] = np.where(winner['Amy'] < winner['Bob'], 'Amy', 'Bob')
winner.style.highlight_min(axis=1)

In [ ]:
winner.Winner.value_counts()

## There's more...

In [ ]:
week4a = week4.copy()
month_chron = week4a['Month'].unique() # or month.drop_duplicates
month_chron

In [ ]:
week4a['Month'] = pd.Categorical(week4a['Month'], 
                                 categories=month_chron,
                                 ordered=True)
week4a.pivot(index='Month', columns='Name', values='Perc Weight Loss')

# Calculating weighted mean SAT scores per state with apply

In [ ]:
college = pd.read_csv('data/college.csv')
subset = ['UGDS', 'SATMTMID', 'SATVRMID']
college2 = college.dropna(subset=subset)
college.shape

In [ ]:
college2.shape

In [ ]:
def weighted_math_average(df):
    weighted_math = df['UGDS'] * df['SATMTMID']
    return int(weighted_math.sum() / df['UGDS'].sum())

In [ ]:
college2.groupby('STABBR').apply(weighted_math_average).head()

In [ ]:
college2.groupby('STABBR').agg(weighted_math_average).head()

In [ ]:
college2.groupby('STABBR')['SATMTMID'].agg(weighted_math_average)

In [ ]:
from collections import OrderedDict
def weighted_average(df):
    data = OrderedDict()
    weight_m = df['UGDS'] * df['SATMTMID']
    weight_v = df['UGDS'] * df['SATVRMID']

    data['weighted_math_avg'] = weight_m.sum() / df['UGDS'].sum()
    data['weighted_verbal_avg'] = weight_v.sum() / df['UGDS'].sum()
    data['math_avg'] = df['SATMTMID'].mean()
    data['verbal_avg'] = df['SATVRMID'].mean()
    data['count'] = len(df)
    return pd.Series(data, dtype='int')

college2.groupby('STABBR').apply(weighted_average).head(10)

In [ ]:
from collections import OrderedDict
def weighted_average(df):
    data = OrderedDict()
    weight_m = df['UGDS'] * df['SATMTMID']
    weight_v = df['UGDS'] * df['SATVRMID']

    wm_avg = weight_m.sum() / df['UGDS'].sum()
    wv_avg = weight_v.sum() / df['UGDS'].sum()

    data['weighted_math_avg'] = wm_avg
    data['weighted_verbal_avg'] = wv_avg
    data['math_avg'] = df['SATMTMID'].mean()
    data['verbal_avg'] = df['SATVRMID'].mean()
    data['count'] = len(df)
    return pd.Series(data, dtype='int')

college2.groupby('STABBR').apply(weighted_average).head(10)

## There's more...

In [ ]:
from scipy.stats import gmean, hmean
def calculate_means(df):
    df_means = pd.DataFrame(index=['Arithmetic', 'Weighted', 'Geometric', 'Harmonic'])
    cols = ['SATMTMID', 'SATVRMID']
    for col in cols:
        arithmetic = df[col].mean()
        weighted = np.average(df[col], weights=df['UGDS'])
        geometric = gmean(df[col])
        harmonic = hmean(df[col])
        df_means[col] = [arithmetic, weighted, geometric, harmonic]
        
    df_means['count'] = len(df)
    return df_means.astype(int)

college2.groupby('STABBR').filter(lambda x: len(x) != 1).groupby('STABBR').apply(calculate_means).head(10)

# Grouping by continuous variables

In [ ]:
flights = pd.read_csv('data/flights.csv')
flights.head()

In [ ]:
flights.DIST.hasnans

In [ ]:
flights.dropna(subset=['DIST']).shape

In [ ]:
bins = [-np.inf, 200, 500, 1000, 2000, np.inf]
cuts = pd.cut(flights['DIST'], bins=bins)
cuts.head()

In [ ]:
cuts.value_counts()

In [ ]:
flights.groupby(cuts)['AIRLINE'].value_counts(normalize=True).round(3).head(15)

## How it works...

In [ ]:
flights.groupby(cuts)['AIRLINE'].value_counts(normalize=True)

## There's more...

In [ ]:
flights.groupby(cuts)['AIR_TIME'].quantile(q=[.25, .5, .75]).div(60).round(2)

In [ ]:
labels=['Under an Hour', '1 Hour', '1-2 Hours', '2-4 Hours', '4+ Hours']
cuts2 = pd.cut(flights['DIST'], bins=bins, labels=labels)
flights.groupby(cuts2)['AIRLINE'].value_counts(normalize=True).round(3).unstack().style.highlight_max(axis=1)

# Counting the total number of flights between cities

In [11]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [12]:
flights_ct = flights.groupby(['ORG_AIR', 'DEST_AIR']).size()
flights_ct.head()

ORG_AIR  DEST_AIR
ATL      ABE         31
         ABQ         16
         ABY         19
         ACY          6
         AEX         40
dtype: int64

In [13]:
flights_ct.loc[[('ATL', 'IAH'), ('IAH', 'ATL')]]

ORG_AIR  DEST_AIR
ATL      IAH         121
IAH      ATL         148
dtype: int64

In [14]:
flights_sort = flights[['ORG_AIR', 'DEST_AIR']].apply(sorted, axis=1)
flights_sort.head()

0    [LAX, SLC]
1    [DEN, IAD]
2    [DFW, VPS]
3    [DCA, DFW]
4    [LAX, MCI]
dtype: object

In [15]:
rename_dict = {'ORG_AIR':'AIR1','DEST_AIR':'AIR2'}
flights_sort = flights_sort.rename(columns=rename_dict)
flights_ct2 = flights_sort.groupby(['AIR1', 'AIR2']).size()
flights_ct2.head()

KeyError: 'AIR1'

In [ ]:
flights_ct2.loc[('ATL', 'IAH')]

In [ ]:
flights_ct2.loc[('IAH', 'ATL')]

## There's more...

In [ ]:
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
data_sorted[:10]

In [ ]:
flights_sort2 = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])
fs_orig = flights_sort.rename(columns={'ORG_AIR':'AIR1', 'DEST_AIR':'AIR2'})
flights_sort2.equals(fs_orig)

In [ ]:
%timeit flights_sort = flights[['ORG_AIR', 'DEST_AIR']].apply(sorted, axis=1)

In [ ]:
%%timeit
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
flights_sort2 = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])

# Finding the longest streak of on-time flights

In [ ]:
s = pd.Series([1, 1, 1, 0, 1, 1, 1, 0])
s

In [ ]:
s1 = s.cumsum()
s1

In [ ]:
s.mul(s1).diff()

In [ ]:
s.mul(s1).diff().where(lambda x: x < 0)

In [ ]:
s.mul(s1).diff().where(lambda x: x < 0).ffill().add(s1, fill_value=0)

In [ ]:
flights = pd.read_csv('data/flights.csv')
flights['ON_TIME'] = flights['ARR_DELAY'].lt(15).astype(int)
flights[['AIRLINE', 'ORG_AIR', 'ON_TIME']].head(10)

In [ ]:
def max_streak(s):
    s1 = s.cumsum()
    return s.mul(s1).diff().where(lambda x: x < 0) \
            .ffill().add(s1, fill_value=0).max()

In [ ]:
flights.sort_values(['MONTH', 'DAY', 'SCHED_DEP']) \
       .groupby(['AIRLINE', 'ORG_AIR'])['ON_TIME'] \
       .agg(['mean', 'size', max_streak]).round(2).head()

## There's more...

In [ ]:
def max_delay_streak(df):
    df = df.reset_index(drop=True)
    s = 1 - df['ON_TIME']
    s1 = s.cumsum()
    streak = s.mul(s1).diff().where(lambda x: x < 0) \
              .ffill().add(s1, fill_value=0)
    last_idx = streak.idxmax()
    first_idx = last_idx - streak.max() + 1
    df_return = df.loc[[first_idx, last_idx], ['MONTH', 'DAY']]
    df_return['streak'] = streak.max()
    df_return.index = ['first', 'last']
    df_return.index.name='streak_row'
    return df_return

In [ ]:
flights.sort_values(['MONTH', 'DAY', 'SCHED_DEP']) \
       .groupby(['AIRLINE', 'ORG_AIR']) \
       .apply(max_delay_streak) \
       .sort_values(['streak','MONTH','DAY'], ascending=[False, True, True]).head(10)